### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [9]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [14]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model



In [15]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [16]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [17]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/py

167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 503us/step - accuracy: 0.7364 - loss: 0.5513
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step - accuracy: 0.8092 - loss: 0.4257
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 946us/step - accuracy: 0.7458 - loss: 0.5527
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 278us/step - accuracy: 0.8180 - loss: 0.4157
125/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6052 - loss: 0.6782Epoch 4/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7667 - loss: 0.5337
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4613 - loss: 0.7917
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6297 - loss: 0.6581
Epoch 2/50
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6580 - loss: 0.6026
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6103 - loss: 0.6709
Epoch 2/50
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7641 - loss: 0.5393
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - accuracy: 0.8

/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/py

167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step - accuracy: 0.8581 - loss: 0.3295
Epoch 48/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step - accuracy: 0.8685 - loss: 0.3271
Epoch 50/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step - accuracy: 0.8586 - loss: 0.3393
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 564us/step - accuracy: 0.8613 - loss: 0.3306
Epoch 48/50
Epoch 49/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step - accuracy: 0.8618 - loss: 0.3301
Epoch 49/50
  1/167 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.9375 - loss: 0.2143Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.8657 - loss: 0.3226
Epoch 47/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step - accuracy: 0.8668 - loss: 0.3067
  1/167 ━━━━━━━━━━━━━━━━━━━━ 33s 204ms/step - accuracy: 0.9688 - loss: 0.1635Epoch 50/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8629 - loss: 0.3281
Epoch 49/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8692 - loss: 0.3201
Epoch 50/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 269us/st

...

 67/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8704 - loss: 0.2923Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8688 - loss: 0.3105
Epoch 26/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8526 - loss: 0.3583
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8723 - loss: 0.2958
Epoch 21/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8830 - loss: 0.2640
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8690 - loss: 0.3057
Epoch 20/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8732 - loss: 0.3067
Epoch 18/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - accuracy: 0.8606 - loss: 0.3224
Epoch 27/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - accuracy: 0.8738 - loss: 0.2929
Epoch 22/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step - accuracy: 0.8650 - loss: 0.3380
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step - accuracy: 0.8702 - loss: 0.3118
Epoch 19/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/ste

/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - accuracy: 0.8665 - loss: 0.3127
Epoch 22/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - accuracy: 0.8725 - loss: 0.3069
Epoch 20/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step - accuracy: 0.8674 - loss: 0.3133
Epoch 29/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - accuracy: 0.8703 - loss: 0.3122
Epoch 8/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - accuracy: 0.8769 - loss: 0.2908
Epoch 24/100
Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step - accuracy: 0.8809 - loss: 0.2902
Epoch 21/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8836 - loss: 0.2971
Epoch 23/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8700 - loss: 0.3063
Epoch 30/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8659 - loss: 0.3258
Epoch 9/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8734 - loss: 0.3060 
Epoch 22/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8697 - loss: 0.2933
Epoch 25/100
167/167 ━

/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/siidh/Documents/DeepLearning_endtoend/venv_name/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 208us/step - accuracy: 0.6784 - loss: 0.6466
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - accuracy: 0.7944 - loss: 0.4723
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 249us/step - accuracy: 0.8072 - loss: 0.4369
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 188us/step - accuracy: 0.8092 - loss: 0.4327
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 239us/step - accuracy: 0.8160 - loss: 0.4191
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 225us/step - accuracy: 0.8232 - loss: 0.4092
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 185us/step - accuracy: 0.8288 - loss: 0.4044
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step - accuracy: 0.8270 - loss: 0.4033
Epoch 9/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 189us/step - accuracy: 0.8411 - loss: 0.3864
Epoch 10/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 231us/step - accuracy: 0.8428 - loss: 0.3735
Epoch 11/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 232us/step - accuracy: 0.8438 - loss: 0.3728
Epoch 12/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 